based on the following link: https://datascienceplus.com/long-short-term-memory-lstm-and-how-to-implement-lstm-using-python/

In [2]:
#Import the libraries
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [3]:
# creating a random array of int's
features = (np.random.randint(10, size=(100, 1)))
print(features.shape)

(100, 1)


In [5]:
# split the data
training_dataset_length = math.ceil(len(features) * .75)
print(training_dataset_length)

75


Preprocess the data i.e feature scaling to scale the data to be valued in between 0 and 1, which is a good practice to scale the data before feeding it into a neural network for optimal performance.

In [6]:
#Scale the all of the data to be values between 0 and 1 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(features)

In [7]:
train_data = scaled_data[0:training_dataset_length  , : ]

#Splitting the data
x_train=[]
y_train = []

for i in range(10, len(train_data)):
    x_train.append(train_data[i-10:i,0])
    y_train.append(train_data[i,0])


In [11]:
#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

#Reshape the data into 3-D array
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))


Building the architecture
* Make an object of the sequential model. Then add the LSTM layer with parameters (units: the dimension of output space, input_shape: the shape of the training set, return_sequences: Ture or False, determines whether to return the last output in the output sequence or the full sequence.
* We add 4 of the LSTM layers each with a dropout layer of value(0.2).{Droupout layer is a type of regularization technique which is used to prevent overfitting, but it may also increase training time in some cases.}
* Final layer is the output layer which is a fully connected dense layer(units = 1, as we are predicting only one value i.e l+1).{Dense layer performs the operation on the input layers and returns the output and every neuron at the previous layer is connected to the neurons in the next layer hence it is called fully connected Dense layer.}

In [13]:
from keras.layers import Dropout

# Initialising the RNN
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model.add(Dense(units = 1))

We will use adam optimizer and the error is calculated by loss function 'mean squared error"

In [14]:
#compile and fit the model on 30 epochs
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 30, batch_size = 50)

Epoch 1/30
2/2 [==============================] - 0s 14ms/step - loss: 0.3567
Epoch 2/30
2/2 [==============================] - 0s 15ms/step - loss: 0.2862
Epoch 3/30
2/2 [==============================] - 0s 12ms/step - loss: 0.2161
Epoch 4/30
2/2 [==============================] - 0s 14ms/step - loss: 0.1379
Epoch 5/30
2/2 [==============================] - 0s 14ms/step - loss: 0.0884
Epoch 6/30
2/2 [==============================] - 0s 20ms/step - loss: 0.0996
Epoch 7/30
2/2 [==============================] - 0s 15ms/step - loss: 0.1144
Epoch 8/30
2/2 [==============================] - 0s 16ms/step - loss: 0.0883
Epoch 9/30
2/2 [==============================] - 0s 15ms/step - loss: 0.0880
Epoch 10/30
2/2 [==============================] - 0s 13ms/step - loss: 0.0803
Epoch 11/30
2/2 [==============================] - 0s 17ms/step - loss: 0.0899
Epoch 12/30
2/2 [==============================] - 0s 13ms/step - loss: 0.0886
Epoch 13/30
2/2 [==============================] - 0s 18ms/st

We now create testdata simular to the traindata

In [15]:
#Test data set
test_data = scaled_data[training_dataset_length - 10: , : ]

#splitting the x_test and y_test data sets
x_test = []
y_test =  features[training_dataset_length : , : ] 

for i in range(10,len(test_data)):
    x_test.append(test_data[i-10:i,0])
    
#Convert x_test to a numpy array 
x_test = np.array(x_test)

#Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [23]:
#check predicted values
predictions = model.predict(x_test) 
#Undo scaling
predictions = scaler.inverse_transform(predictions)

#Calculate RMSE score
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

3.1660675116795463

!!! The smaller the rmse score the better that the model has performed